In [17]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\catra\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
import json
with open('songs.json', 'r') as json_file:
    data = json.load(json_file)

In [6]:
data[0]

{'songName': 'Hurt',
 'spotifyLink': 'https://open.spotify.com/track/1lo9k4PrxFd5Np4cAxXoKo?si=881892db7eaa4a71',
 'genre': 'Rock',
 'lyrics': "I hurt myself today\nTo see if I still feel\nI focus on the pain\nThe only thing that's real\nThe needle tears a hole\nThe old familiar sting\nTry to kill it all away\nBut I remember everything\nWhat have I become?\nMy sweetest friend\nEveryone I know goes away\nIn the end\nAnd you could have it all\nMy empire of dirt\nI will let you down\nI will make you hurt\nI wear this crown of thorns\nUpon my liar's chair\nFull of broken thoughts\nI cannot repair\nBeneath the stains of time\nThe feelings disappear\nYou are someone else\nI'm still right here\nWhat have I become?\nMy sweetest friend\nEveryone I know goes away\nIn the end\nAnd you could have it all\nMy empire of dirt\nI will let you down\nI will make you hurt\nIf I could start again\nA million miles away\nI would keep myself\nI would find a way",
 'mood': '100stress'}

In [7]:
# convert prediction to value

def pred_to_mh(pred):
    label_mapping = {
        0: "Stressed",
        1: "Depressed",
        2: "Neutral",
        3: "Happy",
        4: "Studious"
    }

    return label_mapping[pred]

result = pred_to_mh(1)

In [8]:
result

'Depressed'

In [10]:
word_list = {
    "Stressed": 
        ["stress", "uneasy", "nervous", "discomfort", "inconvenient", "Anxious", "Exhausting", "Intense", 
         "Panic", "Unmanageable", "Breakdown"],
    
    "Depressed":
        ["Melancholic", "Somber", "Blue", "Gloomy, depression", "rain", "Overwhelming", "Desperate", 
         "Suicidal", "Paralyzed", "Incapacitating", "Unbearable", "Devastated", "cry", "tears", "pain"],
    
    "Neutral":
        ["bored", "simple", "pleasant", "normal", "neutral"],
        
    "Happy":
        ["Stable", "Calm", "Satisfied", "Happy", "Content", "Manic", "Euphoric", "Ecstatic", "Party",
         "Dance", "excited"],
        
    "Studious":
        ["smart", "study", "brain", "school", "grades", "graduation", "homework", "scholar", "work",
         "writing", "teacher", "students", "reading", "intelligent", "education"]
}

In [30]:
# clean (remove stopwords and whatnot)
import re
from nltk.corpus import stopwords

list_of_words = word_list[result]
list_of_song_lyrics = [song_record["lyrics"] for song_record in data]

stop_words = set(stopwords.words("english"))
stop_words.update({'im', 'em', 'on', 'say', 'cant', 'bout', 'about', 
                   'that', 'thats', 'put', 'askin', 'weve', 'got', 'could'
                   'would', 'let'})

def remove_stopwords(text):
    text_without_newlines = text.replace("\n", " ")
    words = text_without_newlines.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

def clean_all(list):    
    cleaned_lists = []
    for text in list:
        filtered_text_list = text.lower()
        filtered_text_list = re.sub(r'[^\w\s]', '', filtered_text_list)
        filtered_text_list = remove_stopwords(filtered_text_list)
        cleaned_lists.append(filtered_text_list)
    return cleaned_lists

list_of_song_lyrics = clean_all(list_of_song_lyrics)
list_of_song_lyrics

['hurt today see still feel focus pain thing real needle tears hole old familiar sting try kill away remember everything become sweetest friend everyone know goes away end could empire dirt make hurt wear crown thorns upon liars chair full broken thoughts cannot repair beneath stains time feelings disappear someone else still right become sweetest friend everyone know goes away end could empire dirt make hurt could start million miles away would keep would find way',
 'wasted times spent someone else wasnt even half reminiscin felt niscin felt even though life hell seem forget want belong give love pullin gettin sprung aint cause lot dont make run blowin spot cause aint business catchin feelings anyway aint business catchin feelings catchin feelings girls want youre winnin winnin youve beginnin ooh yeah yeah know right talkin talkin hope know dick still option cause ill beat ill beat yeah took time learn way body functions equestrian ride like champion ride baby sex get high without su

In [29]:
clean_all(list_of_words)

['melancholic',
 'somber',
 'blue',
 'gloomy depression',
 'rain',
 'overwhelming',
 'desperate',
 'suicidal',
 'paralyzed',
 'incapacitating',
 'unbearable',
 'devastated',
 'cry',
 'tears',
 'pain']

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def embed_text(list_of_lists):
    encoded_texts = []

    for sublist in list_of_lists:
        encoded_sublist = []

        for text in sublist:
            encoded_text = tokenizer(text, padding=True, truncation=True, return_tensors='tf')
            encoded_sublist.append(encoded_text)

        encoded_texts.append(encoded_sublist)

    word_index = tokenizer.get_vocab()
    return encoded_texts, word_index

In [43]:
embeded_list_of_song_lyrics, song_word_index = embed_text(list_of_song_lyrics)
embeded_list_of_words, words_word_index = embed_text(list_of_song_lyrics)

In [46]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have word embeddings for your words and songs
# Replace these with your actual word embeddings and song embeddings
# list_of_words_embeddings = np.array([word_embed(word) for word in list_of_words])
# song_lyrics_embeddings = [np.array([word_embed(word) for word in song_lyrics]) for song_lyrics in list_of_song_lyrics]

list_avg_embedding = np.mean(embeded_list_of_song_lyrics, axis=0)

similarities = [cosine_similarity([list_avg_embedding], [song_embedding])[0][0] for song_embedding in song_lyrics_embeddings]

best_match_index = np.argmax(similarities)

best_matching_song = data[best_match_index]['songName']

best_matching_song

c:\Users\catra\anaconda3\envs\brainwave_project\lib\site-packages\numpy\core\_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''